# Summarizing Quarterly Data

Importing Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import urllib
import urllib.request
# for vizualizations
%matplotlib inline 
import matplotlib.pyplot as plt
from matplotlib import cm as cm
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
sns.set()
import requests, pickle
import urllib.request
from bs4 import BeautifulSoup #for web scraping
import csv #for writing csv
import logging #for logging
import os
import zipfile
import boto.s3
import sys
from boto.s3.key import Key
import time
import datetime

session = requests.session()

Programmatically downloading the data

In [ ]:
#Downloading Quarterly Data
url = 'https://freddiemac.embs.com/FLoan/Data/download.php'

cookies = {'PHPSESSID':'ep857b7kh3616r5k9dldjjq040'}
r = requests.post(url,cookies=cookies)
content=r.content
soup = BeautifulSoup(content,'lxml')
all_href = soup.find_all('a')
url_list=['https://freddiemac.embs.com/FLoan/Data/']*13
i=0;
for href in all_href:
    if ('?f=historical_data1_Q' in href['href'])and(href['href'][37]>'6' and href['href'][36]<'1'):
        url_list[i]=url_list[i]+href['href']
        i+=1

if (not os.path.exists('as34')):
    os.mkdir('as34')
os.chdir('as34')
for i in range(13):
    r = requests.get(url_list[i],cookies=cookies)
    with open(url_list[i][54:77]+'.zip','wb') as code:
        code.write(r.content)

os.chdir('..')

Unzipping the files

In [ ]:
os.chdir(r'C:\Users\byron\Desktop\Data-Science-Class\hh\as33\as33\as34')
import glob
import shutil
mylist = [f for f in glob.glob("*.zip")]
import os, zipfile

dir_name = r'C:\Users\byron\Desktop\Data-Science-Class\hh\as33\as33\as34'
extension = ".zip"

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        for i in mylist:
            zip_ref = zipfile.ZipFile(i) # create zipfile object
            zip_ref.extractall(dir_name) # extract file to dir
            zip_ref.close() # close file
            os.remove(i) # delete zipped file

Merging the data into one file

In [ ]:
# merging quarterly data
mylist2 = [f for f in glob.glob("*.txt")]
meragefiledir = r'C:\Users\byron\Desktop\Data-Science-Class\hh\as33\as33\as34'
filenames=os.listdir(meragefiledir)
for i in mylist2:
    file=open(i,'w')
    for filename in filenames:
        filepath=meragefiledir+'\\'+filename
        for line in open(filepath):
            file.writelines(line)
        file.write('\n')
    file.close()

Reading in the merged file for preprocessing

In [ ]:
df = pd.read_csv(filename, sep='|')

Naming the columns in the dataset

In [ ]:
df.columns = ["LOAN SEQUENCE NUMBER","MONTHLY REPORTING PERIOD","CURRENT ACTUAL UPB","CURRENT LOAN DELINQUENCY STATUS","LOAN AGE",
               "REMAINING MONTHS TO LEGAL MATURITY","REPURCHASE FLAG","MODIFICATION FLAG","ZERO BALANCE CODE","ZERO BALANCE EFFECTIVE DATE",
               "CURRENT INTEREST RATE","CURRENT DEFERRED UPB","DUE DATE OF LAST PAID INSTALLMENT (DDLPI)","MI RECOVERIES","NET SALES PROCEEDS",
               "NON MI RECOVERIES","EXPENSES","LEGAL COSTS","MAINTENANCE AND PRESERVATION COSTS","TAXES AND INSURANCE","MISCELLANEOUS EXPENSES",
               "ACTUAL LOSS CALCULATION","MODIFICATION COST","STEP MODIFICATION FLAG","DEFERRED PAYMENT MODIFICATION","ESTIMATED LOAN TO VALUE (ELTV)"]

Dealing with missing values and dropping useless columns

In [ ]:
def drop_missing_values(df):
    df.drop('REPURCHASE FLAG',axis=1,inplace=True)
    df.drop('LOAN SEQUENCE NUMBER',axis=1,inplace=True)
    df.drop('MONTHLY REPORTING PERIOD',axis=1,inplace=True)
    df.drop('MODIFICATION FLAG',axis=1,inplace=True)
    df.drop('ZERO BALANCE CODE',axis=1,inplace=True)
    df.drop('ZERO BALANCE EFFECTIVE DATE',axis=1,inplace=True)
    df.drop('DUE DATE OF LAST PAID INSTALLMENT (DDLPI)',axis=1,inplace=True)
    df.drop('MI RECOVERIES',axis=1,inplace=True)
    df.drop('NET SALES PROCEEDS',axis=1,inplace=True)
    df.drop('NON MI RECOVERIES',axis=1,inplace=True)
    df.drop('EXPENSES',axis=1,inplace=True)
    df.drop('LEGAL COSTS',axis=1,inplace=True)
    df.drop('MAINTENANCE AND PRESERVATION COSTS',axis=1,inplace=True)
    df.drop('TAXES AND INSURANCE',axis=1,inplace=True)
    df.drop('MISCELLANEOUS EXPENSES',axis=1,inplace=True)
    df.drop('ACTUAL LOSS CALCULATION',axis=1,inplace=True)
    df.drop('MODIFICATION COST',axis=1,inplace=True)
    df.drop('STEP MODIFICATION FLAG',axis=1,inplace=True)
    df.drop('ESTIMATED LOAN TO VALUE (ELTV)',axis=1,inplace=True)
    df['DELINQUENT']=['N' if x == '0' else 'Y' for x in df['CURRENT LOAN DELINQUENCY STATUS']]
    df.drop('DEFERRED PAYMENT MODIFICATION',axis=1,inplace=True)
    df.drop('CURRENT LOAN DELINQUENCY STATUS',axis=1,inplace=True)
    df.drop('CURRENT DEFERRED UPB', axis=1, inplace=True)
    df.drop('DELINQUENT', axis=1, inplace=True)
    #df['REPORTING YEAR'] =  [int(str(d)[:4]) for d in df['MONTHLY REPORTING PERIOD']]
    return df

Accessing the preprocessed file

In [ ]:
val = drop_missing_values(df)

Printing summarizing and saving the summary file in a text file

In [ ]:
with open("Summarized_Quarterly_data_file.txt", "w") as text_file:
    print(val.describe(), file=text_file)